# Sentinel-2 Data Preprocessing
## Clipping Sentinel-2 Images around PEP725 Stations

**Inputs**
- parent directory of a S2 image (.SAFE folder)
- `envelope_gdf` (for now I will also include the code to go from buffer to envelope here)

## TODO
- Make the code iterate over different folders. Currently it accesses just a single satellite image
- Look into making the code work without extracting the .zip file of the satellite image. If not, try Extract .zip folder --> Clip patches --> Delete extracted folder.
- Think about converting the extracted patches to another format, as .jp2 may not be suitable for a training dataset
- Find a way to link elements of the GeoDataFrame with the extracted patch to include them as labels such as the class DBL, EC, M

In [ ]:
# Imports. To be cleaned as well

import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import box
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.coords import BoundingBox
from rasterio.features import geometry_mask
import json
import numpy as np
import glob
path = r'C:\Users\Kostas\Downloads\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\GRANULE\L2A_T32UNB_A009543_20170420T103454\IMG_DATA\R10m\L2A_T32UNB_20170420T103021_B02_10m.jp2'

### ------------ Buffer to envelope -----------------

In [105]:
# Read buffer file
buffers = gpd.read_file(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\pep725_high_count_days\buffers_day_92.geojson')
# Set crs because by default GeoPandas loads it in 4326, whereas it is actually 32632
buffers.set_crs(32632, inplace=True, allow_override=True)

# Create envelopes for the buffers
envelope_series = buffers.geometry.envelope
envelope_series.rename('envelope_geometry', inplace=True)
envelope_gdf = buffers.merge(envelope_series, left_index=True, right_index=True)
envelope_gdf = envelope_gdf.drop(['geometry'], axis=1).set_geometry('envelope_geometry').rename_geometry('geometry')

# Change the envelope to a list to use it later
envelope_list = envelope_gdf.geometry.tolist()
# Creating a list of tuples that will be used to preserve the indexing information of the GeoDataFrame.
# This may be of use later, to get information from the GeoDataFrame and put it in the image, e.g., a label such as the class (DBL, EC, M).
envelope_list_with_index = []
for index, row in envelope_gdf.iterrows():
    envelope_list.append((index, row['geometry']))


## Define functions to be used

In [3]:
"""Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""

def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [23]:
'''
This function reads the envelope list and a raster, checks if the polygons are fully contained in the raster 
and returns 4 lists with the boundary coordinates for all the envelopes that are fully contained in the raster.
'''

def getContainedEnvelopeCoords (raster, envelope_list):
    with rasterio.open(raster, driver='JP2OpenJPEG') as src:
        raster_extent = src.bounds
        
        # List initialization
        minx_list = []
        miny_list = []
        maxx_list = []
        maxy_list = []

        for poly in envelope_list:
            poly_extent = poly.bounds

            # Check if the polygon is fully inside the raster's extent
            if (poly_extent[0] >= raster_extent[0] and poly_extent[2] <= raster_extent[2] and
                poly_extent[1] >= raster_extent[1] and poly_extent[3] <= raster_extent[3]):
                    minx_list.append(poly_extent[0])
                    miny_list.append(poly_extent[1])
                    maxx_list.append(poly_extent[2])
                    maxy_list.append(poly_extent[3])
    return minx_list, miny_list, maxx_list, maxy_list
        

In [5]:
'''
This function receives a raster file (.jp2) and the boundary coordinates for a polygon. 
It then clips the raster to the extent of the polygon. 
The polygon has to intersect the raster for the operation to be completed
'''

def exportImage(raster, output_path, minx, miny, maxx, maxy):
    # open the raster file (Single Band)
    data = rasterio.open(raster, driver='JP2OpenJPEG')

    # Create a bounding box from the polygon min-max coordinates    
    bbox = box(minx, miny, maxx, maxy)
    # Create a geodataframe with a single polygon so that it can be used with rasterio
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs='32632')
    # Transform the geodataframe to a GeoJSON-like object that can be used as an input in the rasterio mask function
    coords = getFeatures(geo)
    #print(coords)
    
    # Mask and crop the raster AOI where polygon overlaps the whole raster
    out_img, out_transform = mask(data, shapes=coords, crop=True)
    # Define resolution and more
    out_profile = data.profile.copy()
    
    out_profile.update({'width': out_img.shape[2],'height': out_img.shape[1], 'transform': out_transform})
    # Write the extracted raster patch to a file
    with rasterio.open(output_path, 'w', **out_profile) as dst:
        dst.write(out_img)
    
    data.close()
    data = None

In [ ]:
'''
This function is used to receive a string from the raster file's name
using the split() method. It splits the string wherever an underscore appears and then accesses the second-to-last element.
'''
def imageNaming(raster_path):
    string_parts = raster_path.split("_")
    band_string = string_parts[-3]+ "_" + string_parts[-2]
    return band_string

# Example output: 20170420T103021_B02_10m

## Getting a list of all the band rasters

In [6]:
# Make a list of all bands in an S2 image from the images' .SAFE folder (.jp2 files)

dirr = r'C:\Users\Kostas\Downloads\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE'

# This will do it for just the bands 01-12. Other products are omitted.
jp2_files = glob.glob(dirr + '/**/IMG_DATA/**/R??m/*B??_??m.jp2', recursive=True)
print(jp2_files)

# This will provide all .jp2 files in every subfolder of the dirr
#jp2_files = glob.glob(dirr+"/**/*.jp2", recursive=True) 

['C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B02_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B03_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B04_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B08_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R20m\\L2A_T32UNB_20170420T103021_B02_20m.jp2',

## Patch extraction. All bands.

In [21]:
output_path = r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\pep725_high_count_days\rasters'

# Get the boundaries in minx, miny, maxx, maxy and convert them to a list
# bounds = clipped_gdf.geometry.apply(lambda x : x.bounds).tolist()

NameError: name 'clipped_gdf' is not defined

In [100]:
# Need to add a print statement that shows how many polygons are contained

# Read each raster file from the jp2_files list
for raster in range(0, len(jp2_files)):

    # Get the 4 lists of the minx, miny, maxx, maxy for the polygons fully contained in the raster
    minx_list, miny_list, maxx_list, maxy_list = getContainedEnvelopeCoords(jp2_files[raster], envelope_list)
    
    # Get just the name of the raster (Date_Time_Band) to use it in naming the extracted images
    raster_name = imageNaming(jp2_files[raster])
    print("Creating images around PEP725 stations for the file:", raster_name + ".jp2")

    # Now, for the raster at hand, iterate over each polygon and call the exportImage function to extract a patch for each polygon
    for i in range(0, len(minx_list)):
        # Name the patch
        output_file_name = os.path.join(output_path, raster_name + f"_Patch_{i+1}.jp2")
        # Extract it
        exportImage(jp2_files[raster], output_file_name, minx_list[i], miny_list[i], maxx_list[i], maxy_list[i])

Creating images around PEP725 stations for the file: 20170420T103021_B02.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B03.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B04.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B08.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B02.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B03.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B04.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B05.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B06.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B07.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B11.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B12.jp2
Creating images around PEP725 stations for the file: 20170420T103021_B8A.jp2